```{contents}
```
## Quantizations

Below is a **clear, complete, LLM-focused explanation** of Quantization and the four major methods used today: **8-bit, 4-bit, AWQ, GPTQ**.
This is the exact level used in LLM engineering interviews and production documentation.

---

### **What Is Quantization in LLMs?**

Quantization is the process of **reducing the precision** of model weights (and sometimes activations) from high-bit formats (FP32, FP16, BF16) to **lower-bit formats like INT8 or INT4**.

### **Goal:**

* Reduce **memory usage**
* Reduce **VRAM requirements**
* Increase **inference speed**
* Allow LLMs to run on smaller hardware (like single GPUs/CPUs)

### **What Changes?**

Weights normally stored as FP16 (16-bit floating point):

```
FP16 (16 bits) → INT8 (8 bits) → INT4 (4 bits)
```

Lower bits → smaller, faster model.

---

### Why Quantization is Critical for LLMs

Example: A 70B model in FP16:

```
70 billion params × 2 bytes ≈ 140 GB VRAM
```

Impossible on normal hardware.

Quantization reduces memory:

| Format                         | Size reduction              |
| ------------------------------ | --------------------------- |
| **8-bit**                      | ~2× smaller                 |
| **4-bit**                      | ~4× smaller                 |
| **GPTQ / AWQ optimized 4-bit** | Very close to FP16 accuracy |

This is why models like LLaMA-13B can run on a **single 8GB GPU** using 4-bit quantization.

---

### **8-bit Quantization**


#### **What It Is**

Weights are stored using **8 bits (INT8)** rather than FP16.

#### **Pros**

* 2x smaller model
* Fast inference
* Minimal accuracy drop
* Supported natively in Transformers, bitsandbytes

#### **Cons**

* Still too big for large models (e.g., 70B)
* Only weights quantized (not activations)

#### **Where Used**

* bitsandbytes `load_in_8bit=True`
* Intel/NVIDIA INT8 inference

---


### **4-bit Quantization**


#### **What It Is**

Weights stored in **4 bits (INT4 / NF4)**.

4-bit techniques include:

* **NF4 (Normalized Float 4)**
* **FP4**
* **INT4**

#### **Pros**

* 4× smaller model
* Enables 13B, 33B, 70B LLMs on consumer GPUs
* Good accuracy when using NF4 (special distribution)

#### **Cons**

* More accuracy drop than INT8
* Some hardware struggles with pure INT4 math

#### **Where Used**

* QLoRA uses **NF4 quantization**
* bitsandbytes `load_in_4bit=True`

---


### **GPTQ (GPT Quantization)**


A **post-training** quantization method designed specifically for LLMs.

#### **What GPTQ Does**

* Quantizes **each layer optimally**
* Minimizes the **error between quantized and original output**
* Uses a calibration dataset to match full-precision outputs
* Highly optimized for **4-bit inference**

#### **Pros**

* Best 4-bit inference quality
* Fast and accurate
* Supports very large models (70B+)
* Used in HuggingFace `.gptq` models

#### **Cons**

* Offline quantization step required
* Slower to quantize, but fast to run

#### **Where Used**

* LM Studio
* text-generation-webui
* ExLlama, ExLlamaV2 engines
* HuggingFace GPTQ models

---


### **AWQ (Activation-aware Weight Quantization)**


AWQ quantizes **only the weights**, but keeps activations high precision.
It focuses on **preserving important weights**.

#### **Key Idea**

* Identify important channels/weights
* Leave those unquantized (or less quantized)
* Quantize unimportant weights to INT4

#### **Pros**

* High accuracy at 4-bit
* Better than GPTQ for some models
* Very fast inference (supported by vLLM + AWQ)
* Stable for extremely large models

#### **Cons**

* Slightly larger memory than pure 4-bit GPTQ
* Not as widely supported (except vLLM, AWQ libraries)

#### **Where Used**

* vLLM AWQ
* llama.cpp (partial)
* Commercial inference (AWS, Together, Mosaic)

---


### **Accuracy Comparison**


| Method      | Bits | Speed   | Accuracy     | Notes                |
| ----------- | ---- | ------- | ------------ | -------------------- |
| FP16        | 16   | Slow    | 🔥 Best      | Baseline             |
| INT8        | 8    | Faster  | 🔥 Very high | Good trade-off       |
| NF4 (QLoRA) | 4    | Fast    | ⭐ High       | For fine-tuning      |
| GPTQ 4-bit  | 4    | Fastest | ⭐⭐ High+     | For inference        |
| AWQ 4-bit   | 4    | Fastest | ⭐⭐⭐ Highest  | Best for LLM serving |

---


### **Which One to Use? (Short Guide)**


#### **For inference (running a model)**

| Hardware                 | Best choice           |
| ------------------------ | --------------------- |
| GPU (NVIDIA)             | **GPTQ or AWQ 4-bit** |
| CPU                      | **GGUF / llama.cpp**  |
| Apple Silicon (M1/M2/M3) | **GGUF (Q4, Q5)**     |
| Multi-GPU cluster        | **AWQ with vLLM**     |

---

#### **For fine-tuning**

| Goal                         | Best choice     |
| ---------------------------- | --------------- |
| Full-scale fine-tuning       | FP16/BF16       |
| Cheap supervised fine-tuning | **QLoRA (NF4)** |
| Parameter-efficient tuning   | LoRA + 4-bit    |

---


### **Mini Example: Load a 4-bit LLM**


```python
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "TheBloke/Llama-2-7B-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True
)
```

This loads a **GPTQ-quantized 7B LLM** in ~4-bit precision.

---


### **Mini Example: QLoRA with 4-bit NF4**


```python
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    quantization_config={"bnb_4bit_compute_dtype": "bfloat16"}
)
```

This loads the LLM in **4-bit NF4** for fine-tuning.

---

**Final Summary**

| Method                | Purpose                    | Best For                        |
| --------------------- | -------------------------- | ------------------------------- |
| **INT8**              | Smaller model, stable      | General LLM inference           |
| **4-bit (NF4 / FP4)** | Very small model           | QLoRA fine-tuning               |
| **GPTQ**              | Layer-wise optimized quant | Fast inference                  |
| **AWQ**               | Activation-aware quant     | High-accuracy 4-bit LLM serving |

### Demonstration

* Loads the same causal LLM in different precisions (if supported on your machine),
* Runs a small set of prompts,
* Measures **inference latency** and **GPU memory usage** (when a CUDA device is present),
* Prints generated outputs so you can compare quality.

The script is defensive: it attempts each loading method only if the required libraries are installed and the hardware supports it, and it falls back gracefully otherwise.

> **Requirements**
>
> * Python 3.8+
> * `transformers`, `torch` (with CUDA if you have GPU), optionally:
>
>   * `bitsandbytes` (for 8-bit): `pip install bitsandbytes`
>   * `auto-gptq` (for GPTQ 4-bit): `pip install auto-gptq`
>   * `accelerate` (helpful): `pip install accelerate`
> * A GPU is strongly recommended for meaningful speed/memory differences. The script will still run on CPU but INT8/4-bit tests may not be available.

Save as `compare_precisions.py` and run: `python compare_precisions.py`.

```python
"""
compare_precisions.py

Demonstrate FP16 vs INT8 vs GPTQ(4-bit) inference: latency, memory, and outputs.

Usage:
    python compare_precisions.py

Notes:
 - The script tries three loading methods and skips unsupported cases.
 - For INT8 it uses bitsandbytes via transformers (load_in_8bit=True).
 - For GPTQ it uses auto-gptq (AutoGPTQForCausalLM), if available.
 - Replace MODEL_NAME with a model you have access to (HuggingFace).
"""

import time
import os
import sys
import json

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Optional imports (may not be installed)
_try_bitsandbytes = True
_try_auto_gptq = True
try:
    import bitsandbytes as bnb  # noqa: F401
except Exception:
    _try_bitsandbytes = False

try:
    from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig  # noqa: F401
except Exception:
    _try_auto_gptq = False

# ------ Config ------
MODEL_NAME = os.environ.get("MODEL_NAME", "facebook/opt-1.3b")  # change as needed
PROMPTS = [
    "Explain the greenhouse effect in simple terms.",
    "Write a 2-line poem about coffee and coding.",
]
MAX_NEW_TOKENS = 64
TEMPERATURE = 0.2
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
WARMUP = 1  # warmup runs to stabilize timing
# --------------------

def print_header(title):
    print("\n" + "=" * 80)
    print(title)
    print("=" * 80)

def measure_inference(model, tokenizer, prompts, device, max_new_tokens=64, temperature=0.2):
    """
    Runs generation for each prompt, returns timing (avg per prompt), outputs, and memory stats.
    Measures GPU memory if device == 'cuda'.
    """
    model.eval()
    outputs = []
    latencies = []
    # clear cache stats
    if device == "cuda":
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.empty_cache()
    # warmup
    for _ in range(WARMUP):
        for p in prompts[:1]:
            tok = tokenizer(p, return_tensors="pt").to(device)
            with torch.no_grad():
                _ = model.generate(**tok, max_new_tokens=8, do_sample=False)

    for p in prompts:
        tok = tokenizer(p, return_tensors="pt").to(device)
        torch.cuda.synchronize() if device == "cuda" else None
        t0 = time.time()
        with torch.no_grad():
            out = model.generate(**tok, max_new_tokens=max_new_tokens, temperature=temperature)
        torch.cuda.synchronize() if device == "cuda" else None
        t1 = time.time()
        latencies.append(t1 - t0)
        decoded = tokenizer.decode(out[0], skip_special_tokens=True)
        outputs.append(decoded)
    avg_latency = sum(latencies) / len(latencies)
    mem_stats = {}
    if device == "cuda":
        mem_stats["max_alloc_mb"] = torch.cuda.max_memory_allocated() / 1024**2
        mem_stats["reserved_mb"] = torch.cuda.max_memory_reserved() / 1024**2 if hasattr(torch.cuda, "max_memory_reserved") else None
    else:
        # on CPU, report process memory usage if psutil available
        try:
            import psutil
            p = psutil.Process(os.getpid())
            mem_stats["rss_mb"] = p.memory_info().rss / 1024**2
        except Exception:
            mem_stats["rss_mb"] = None
    return {"avg_latency_s": avg_latency, "outputs": outputs, "mem": mem_stats}

def load_fp16(model_name, device):
    """Load FP16 (or BF16) model - best accuracy. Requires sufficient GPU memory."""
    print_header(f"Loading FP16 model '{model_name}' to {device}")
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    # Try loading in half precision if device is CUDA
    if device == "cuda":
        model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
    else:
        # CPU fallback (will be heavy)
        model = AutoModelForCausalLM.from_pretrained(model_name)
    return model, tokenizer

def load_int8(model_name, device):
    """
    Load an 8-bit model using bitsandbytes (transformers integration).
    Requires bitsandbytes installed and a CUDA device.
    """
    if not _try_bitsandbytes:
        print("bitsandbytes not available; skipping INT8.")
        return None, None
    if device != "cuda":
        print("INT8 via bitsandbytes requires CUDA device; skipping INT8.")
        return None, None
    print_header(f"Loading INT8 (bitsandbytes) model '{model_name}' to CUDA (8-bit)")
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    # transformers supports load_in_8bit
    model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True, device_map="auto")
    return model, tokenizer

def load_gptq(model_name, device):
    """
    Load a GPTQ-quantized model if auto-gptq is available. The path should point to a quantized directory,
    or you can quantize an FP16 model first (not done here).
    Note: AutoGPTQ.from_quantized expects a directory with quantized files.
    """
    if not _try_auto_gptq:
        print("auto-gptq not installed; skipping GPTQ (INT4).")
        return None, None
    # If the user provided a local GPTQ dir via env, use it; otherwise skip.
    gptq_dir = os.environ.get("GPTQ_MODEL_DIR", None)
    if not gptq_dir:
        print("No GPTQ model dir set in GPTQ_MODEL_DIR env var; skipping GPTQ.")
        return None, None
    print_header(f"Loading GPTQ quantized model from '{gptq_dir}'")
    # AutoGPTQ provides from_quantized; use device_map auto
    tokenizer = AutoTokenizer.from_pretrained(gptq_dir, use_fast=True)
    model = AutoGPTQForCausalLM.from_quantized(gptq_dir, device_map="auto", use_safetensors=True)
    return model, tokenizer

def main():
    print_header("Precision comparison demo")
    print(f"Model: {MODEL_NAME}")
    print(f"Device: {DEVICE}")
    print(f"Prompts: {len(PROMPTS)} prompts, max_new_tokens={MAX_NEW_TOKENS}")
    results = {}
    # FP16
    try:
        model_fp16, tok_fp16 = load_fp16(MODEL_NAME, DEVICE)
        res = measure_inference(model_fp16, tok_fp16, PROMPTS, DEVICE, max_new_tokens=MAX_NEW_TOKENS, temperature=TEMPERATURE)
        results["fp16"] = res
        # free memory
        del model_fp16
        if DEVICE == "cuda":
            torch.cuda.empty_cache()
    except Exception as e:
        print("FP16 load or inference failed:", e)
        results["fp16"] = {"error": str(e)}

    # INT8
    try:
        model_int8, tok_int8 = load_int8(MODEL_NAME, DEVICE)
        if model_int8 is not None:
            res = measure_inference(model_int8, tok_int8, PROMPTS, DEVICE, max_new_tokens=MAX_NEW_TOKENS, temperature=TEMPERATURE)
            results["int8"] = res
            del model_int8
            if DEVICE == "cuda":
                torch.cuda.empty_cache()
        else:
            results["int8"] = {"skipped": True}
    except Exception as e:
        print("INT8 load or inference failed:", e)
        results["int8"] = {"error": str(e)}

    # GPTQ (4-bit) - requires GPTQ model directory in GPTQ_MODEL_DIR env var
    try:
        model_gptq, tok_gptq = load_gptq(MODEL_NAME, DEVICE)
        if model_gptq is not None:
            res = measure_inference(model_gptq, tok_gptq, PROMPTS, DEVICE, max_new_tokens=MAX_NEW_TOKENS, temperature=TEMPERATURE)
            results["gptq_4bit"] = res
            del model_gptq
            if DEVICE == "cuda":
                torch.cuda.empty_cache()
        else:
            results["gptq_4bit"] = {"skipped": True}
    except Exception as e:
        print("GPTQ load or inference failed:", e)
        results["gptq_4bit"] = {"error": str(e)}

    # Print summary
    print_header("Summary")
    print(json.dumps(results, indent=2, default=lambda o: str(o)))

    # Print outputs side by side for visual comparison (if available)
    print_header("Outputs comparison")
    for i, p in enumerate(PROMPTS):
        print(f"\nPrompt {i+1}: {p}\n")
        for key in ["fp16", "int8", "gptq_4bit"]:
            entry = results.get(key, {})
            outputs = entry.get("outputs")
            if outputs:
                print(f"--- {key}:")
                print(outputs[i][:1000])  # trim
            else:
                note = entry.get("error") or ("skipped" if entry.get("skipped") else "n/a")
                print(f"--- {key}: {note}")

if __name__ == "__main__":
    main()
```

---

### How to use / adapt the script

* Replace `MODEL_NAME` with a model you can access (HF model id). For a small, fast test use `"gpt2"` or `"facebook/opt-125m"` — but note many small models don't meaningfully show memory differences.
* To test **GPTQ (INT4)**, produce a GPTQ quantized model directory and set `GPTQ_MODEL_DIR=/path/to/quantized_model` in your environment before running the script. (AutoGPTQ quantization step is separate.)
* For **INT8**, ensure `bitsandbytes` is installed and CUDA is available; transformers `load_in_8bit=True` requires bitsandbytes.
* The script measures **average latency** across prompts and **GPU peak allocation** via `torch.cuda.max_memory_allocated()` (MB). On CPU, it attempts to measure RSS via `psutil` if available.
* You can extend the script to compute **perplexity**, **BLEU**, or **embedding similarity** between outputs to quantify quality differences.

---

### Interpretation guidelines

* **FP16**: expected highest quality and highest memory use; baseline latency.
* **INT8**: typically halves memory with minimal accuracy loss; latency usually improves.
* **INT4 / GPTQ**: biggest memory reduction and fastest latency; slight accuracy differences may appear depending on model and prompt.